In [ ]:
"""
Project
This project is a toy example for the ETL pipeline. ETL stands for extraction, transformation, and loading. In this pipeline, 
we extract data from various sources, transform the data with respect to a design (i.e., ERD), and load the transformed data to a database.

This project will involve eight related CSV files.
1. olist_customers_dataset.csv
Customer information (customer_id, zip code, city, state).

2. olist_orders_dataset.csv
Order master table (timestamps, status, customer_id).

3. olist_order_items_dataset.csv
Order → products (seller_id, product_id, freight, price).

4. olist_order_payments_dataset.csv
Payment installments per order (sequential payments).

5. olist_order_reviews_dataset.csv
Customer reviews + review timestamps.

6. olist_products_dataset.csv
Product info (name length, description length, category, weight, size).

7. olist_sellers_dataset.csv
Seller info (seller_id, zip code prefix, city, state).

8. product_category_name_translation.csv
Mapping Portuguese category → English category names.

This project could be broken down into the following tasks:

Manually inspect the files and design a database.
Implement your database design.
Load data from files into database.
Write some basic queries.
"""

In [37]:
"""
Task 1: Connect to the database (Done for you)
We will use JupySQL and sqlalchemy packages for creating a database, tables, and peforming SQL queries via SQL magic funciton.

For processing csv files, we will use pandas, the most popular data manipulation library.
"""

'\nTask 1: Connect to the database (Done for you)\nWe will use JupySQL and sqlalchemy packages for creating a database, tables, and peforming SQL queries via SQL magic funciton.\n\nFor processing csv files, we will use pandas, the most popular data manipulation library.\n'

In [4]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [5]:
engine = create_engine("sqlite:///olist_store.db")

In [6]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [7]:
%sql engine

In [8]:
%sql sqlite:///olist_store.db

In [9]:
"""
Task 3: Implement your database design
We will be ussing DDL to create tables in the database. Use the cells below to add your CREATE TABLE statements. Add extra cells as necessary.
SQLite Data Types: https://www.sqlite.org/datatype3.html
"""

'\nTask 3: Implement your database design\nWe will be ussing DDL to create tables in the database. Use the cells below to add your CREATE TABLE statements. Add extra cells as necessary.\nSQLite Data Types: https://www.sqlite.org/datatype3.html\n'

In [10]:
#### 1: Created Customer table 

In [11]:
%%sql

DROP TABLE IF EXISTS Customers;

CREATE TABLE Customers (
    customer_id TEXT PRIMARY KEY,
    customer_unique_id TEXT,
    customer_zip_code_prefix TEXT,
    customer_city TEXT,
    customer_state CHAR(2)
);


Running query in 'sqlite:///olist_store.db'

++
||
++
++

In [12]:
"""
We can check whether the table is created correctly in various ways. One way is to check the schema.
"""

'\nWe can check whether the table is created correctly in various ways. One way is to check the schema.\n'

In [13]:
%%sql

SELECT sql FROM sqlite_schema WHERE name='Customers';

Running query in 'sqlite:///olist_store.db'

sql
"CREATE TABLE Customers ( customer_id TEXT PRIMARY KEY, customer_unique_id TEXT, customer_zip_code_prefix TEXT, customer_city TEXT, customer_state CHAR(2))"


In [14]:
"""
The table should be empty.
"""

'\nThe table should be empty.\n'

In [15]:
%%sql

SELECT count(*) from Customers;

Running query in 'sqlite:///olist_store.db'

count(*)
0


In [16]:
#### 2: Created Orders table

In [17]:
%%sql sqlite:///olist_store.db
DROP TABLE IF EXISTS Orders;

CREATE TABLE Orders (
  order_id TEXT PRIMARY KEY,
  customer_id TEXT NOT NULL,
  order_status TEXT,
  order_purchase_timestamp TIMESTAMP,
  order_approved_at TIMESTAMP,
  order_delivered_carrier_date TIMESTAMP,
  order_delivered_customer_date TIMESTAMP,
  order_estimated_delivery_date TIMESTAMP,
  FOREIGN KEY (customer_id) REFERENCES Customers(customer_id)
);


++
||
++
++

In [18]:
%%sql

SELECT count(*) from Orders;

Running query in 'sqlite:///olist_store.db'

count(*)
0


In [19]:
### 1. Customers – `olist_customers_dataset.csv` → `Customers` table
"""
**Goal:** Create a clean customer dimension table with one row per customer.

**Steps:**
- Read the CSV using `pandas.read_csv`.
- Converted `customer_id` and `customer_unique_id` to strings and removed extra spaces using `.astype(str).str.strip()`.
- Standardized city and state formatting:
  - `customer_city` → title case (e.g., "sao paulo" → "Sao Paulo").
  - `customer_state` → uppercase two-letter code.
- Kept `customer_zip_code_prefix` as **TEXT** because values can contain non-numeric codes (e.g., `"ZA"`, `"PM"`), and stripped whitespace.
- Dropped any rows with missing `customer_id` and removed duplicates based on `customer_id` to respect the primary key.
- Loaded the cleaned DataFrame into the `Customers` table in SQLite using `to_sql(..., if_exists="append")`.
"""

'\n**Goal:** Create a clean customer dimension table with one row per customer.\n\n**Steps:**\n- Read the CSV using `pandas.read_csv`.\n- Converted `customer_id` and `customer_unique_id` to strings and removed extra spaces using `.astype(str).str.strip()`.\n- Standardized city and state formatting:\n  - `customer_city` → title case (e.g., "sao paulo" → "Sao Paulo").\n  - `customer_state` → uppercase two-letter code.\n- Kept `customer_zip_code_prefix` as **TEXT** because values can contain non-numeric codes (e.g., `"ZA"`, `"PM"`), and stripped whitespace.\n- Dropped any rows with missing `customer_id` and removed duplicates based on `customer_id` to respect the primary key.\n- Loaded the cleaned DataFrame into the `Customers` table in SQLite using `to_sql(..., if_exists="append")`.\n'

In [20]:
import pandas as pd
from sqlalchemy import create_engine

cust = pd.read_csv(r"C:\Users\kisho\Downloads\Kishor Project\olist_customers_dataset.csv")

# clean
cust["customer_id"] = cust["customer_id"].astype(str).str.strip()
cust["customer_unique_id"] = cust["customer_unique_id"].astype(str).str.strip()
cust["customer_city"] = cust["customer_city"].astype(str).str.strip().str.title()
cust["customer_state"] = cust["customer_state"].astype(str).str.strip().str.upper()
cust["customer_zip_code_prefix"] = cust["customer_zip_code_prefix"].astype(str).str.strip().str.upper()

# PK hygiene
cust = cust.dropna(subset=["customer_id"]).drop_duplicates(subset=["customer_id"])

# load
cust.to_sql("Customers", con=engine, if_exists="append", index=False)
len(cust)


99441

In [21]:
%%sql sqlite:///olist_store.db
SELECT COUNT(*) AS rows FROM Customers;
SELECT * FROM Customers LIMIT 10;


customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,Franca,SP
18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,Sao Bernardo Do Campo,SP
4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,Sao Paulo,SP
b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,Mogi Das Cruzes,SP
4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,Campinas,SP
879864dab9bc3047522c92c82e1212b8,4c93744516667ad3b8f1fb645a3116a4,89254,Jaragua Do Sul,SC
fd826e7cf63160e536e0908c76c3f441,addec96d2e059c80c30fe6871d30d177,4534,Sao Paulo,SP
5e274e7a0c3809e14aba7ad5aae0d407,57b2a98a409812fe9618067b6b8ebe4f,35182,Timoteo,MG
5adf08e34b2e993982a47070956c5c65,1175e95fb47ddff9de6b2b06188f7e0d,81560,Curitiba,PR
4b7139f34592b3a31687243a302fa75b,9afe194fb833f79e300e37e580171f22,30575,Belo Horizonte,MG


In [22]:
### 2. Orders – `olist_orders_dataset.csv` → `Orders` table
"""
**Goal:** Build the main fact table of orders with clean timestamps and valid customer references.

**Steps:**
- Read the CSV into a DataFrame.
- Cleaned identifier and category columns:
  - `order_id` and `customer_id` → strings with `.astype(str).str.strip()`.
  - `order_status` → trimmed and converted to lowercase.
- Converted all date/time columns to proper 24-hour ISO strings:
  - Columns: `order_purchase_timestamp`, `order_approved_at`,
    `order_delivered_carrier_date`, `order_delivered_customer_date`,
    `order_estimated_delivery_date`.
  - Used `pd.to_datetime(..., errors="coerce")` and then `.dt.strftime("%Y-%m-%d %H:%M:%S")` to standardize the format.
- Dropped rows where `order_id`, `customer_id`, or `order_purchase_timestamp` were missing.
- Removed duplicate orders based on `order_id`.
- Enforced foreign key integrity at the data level by keeping only orders whose `customer_id` appears in the `Customers` table.
- Loaded the result into the `Orders` table.
"""

'\n**Goal:** Build the main fact table of orders with clean timestamps and valid customer references.\n\n**Steps:**\n- Read the CSV into a DataFrame.\n- Cleaned identifier and category columns:\n  - `order_id` and `customer_id` → strings with `.astype(str).str.strip()`.\n  - `order_status` → trimmed and converted to lowercase.\n- Converted all date/time columns to proper 24-hour ISO strings:\n  - Columns: `order_purchase_timestamp`, `order_approved_at`,\n    `order_delivered_carrier_date`, `order_delivered_customer_date`,\n    `order_estimated_delivery_date`.\n  - Used `pd.to_datetime(..., errors="coerce")` and then `.dt.strftime("%Y-%m-%d %H:%M:%S")` to standardize the format.\n- Dropped rows where `order_id`, `customer_id`, or `order_purchase_timestamp` were missing.\n- Removed duplicate orders based on `order_id`.\n- Enforced foreign key integrity at the data level by keeping only orders whose `customer_id` appears in the `Customers` table.\n- Loaded the result into the `Orders` tab

In [23]:
import pandas as pd

orders = pd.read_csv(r"C:\Users\kisho\Downloads\Kishor Project\olist_orders_dataset.csv")

orders["order_id"] = orders["order_id"].astype(str).str.strip()
orders["customer_id"] = orders["customer_id"].astype(str).str.strip()
orders["order_status"] = orders["order_status"].astype(str).str.strip().str.lower()

dt_cols = [
    "order_purchase_timestamp",
    "order_approved_at",
    "order_delivered_carrier_date",
    "order_delivered_customer_date",
    "order_estimated_delivery_date",
]
for c in dt_cols:
    orders[c] = pd.to_datetime(orders[c], errors="coerce", infer_datetime_format=True)
    orders[c] = orders[c].dt.strftime("%Y-%m-%d %H:%M:%S")

orders = orders.dropna(subset=["order_id","customer_id","order_purchase_timestamp"])
orders = orders.drop_duplicates(subset=["order_id"])

# keep only customers that exist (FK integrity)
existing = set(pd.read_sql("SELECT customer_id FROM Customers", con=engine)["customer_id"])
orders = orders[orders["customer_id"].isin(existing)]

orders.to_sql("Orders", con=engine, if_exists="append", index=False)
len(orders)


C:\Users\kisho\AppData\Local\Temp\ipykernel_40500\329149739.py:17: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  orders[c] = pd.to_datetime(orders[c], errors="coerce", infer_datetime_format=True)
C:\Users\kisho\AppData\Local\Temp\ipykernel_40500\329149739.py:17: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  orders[c] = pd.to_datetime(orders[c], errors="coerce", infer_datetime_format=True)
C:\Users\kisho\AppData\Local\Temp\ipykernel_40500\329149739.py:17: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version

99441

In [24]:
%%sql sqlite:///olist_store.db
SELECT COUNT(*) AS rows FROM Orders;
SELECT *
FROM Orders
LIMIT 5;


order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [25]:
#### Created Reviwes Table

In [26]:
%%sql sqlite:///olist_store.db
DROP TABLE IF EXISTS Reviews;

CREATE TABLE Reviews (
  review_id TEXT PRIMARY KEY,
  order_id  TEXT NOT NULL,
  review_score INTEGER,
  review_comment_title   TEXT,
  review_comment_message TEXT,
  review_creation_date   TIMESTAMP,
  review_answer_timestamp TIMESTAMP,
  FOREIGN KEY (order_id) REFERENCES Orders(order_id)
);


++
||
++
++

In [27]:
### 8. Reviews – `olist_order_reviews_dataset.csv` → `Reviews` table

"""
**Goal:** Capture customer feedback and ratings linked to orders, with creation and answer timestamps.

**Steps:**
- Read the CSV into pandas.
- Cleaned identifier fields:
  - `review_id` and `order_id` → strings with whitespace stripped.
- Converted `review_score` to numeric Int64.
- Cleaned comment text:
  - `review_comment_title` and `review_comment_message` → converted to pandas string type and trimmed; missing values are kept as NaN so they become NULL in the database.
- Converted timestamp columns:
  - `review_creation_date` and `review_answer_timestamp` → parsed as datetimes and formatted as `YYYY-MM-DD HH:MM:SS`.
- Dropped rows with missing `review_id` or `order_id` and removed duplicate `review_id`s to respect the primary key.
- Enforced FK integrity by keeping only reviews whose `order_id` exists in the `Orders` table.
- Inserted the cleaned DataFrame into the `Reviews` table.
"""

'\n**Goal:** Capture customer feedback and ratings linked to orders, with creation and answer timestamps.\n\n**Steps:**\n- Read the CSV into pandas.\n- Cleaned identifier fields:\n  - `review_id` and `order_id` → strings with whitespace stripped.\n- Converted `review_score` to numeric Int64.\n- Cleaned comment text:\n  - `review_comment_title` and `review_comment_message` → converted to pandas string type and trimmed; missing values are kept as NaN so they become NULL in the database.\n- Converted timestamp columns:\n  - `review_creation_date` and `review_answer_timestamp` → parsed as datetimes and formatted as `YYYY-MM-DD HH:MM:SS`.\n- Dropped rows with missing `review_id` or `order_id` and removed duplicate `review_id`s to respect the primary key.\n- Enforced FK integrity by keeping only reviews whose `order_id` exists in the `Orders` table.\n- Inserted the cleaned DataFrame into the `Reviews` table.\n'

In [28]:
import pandas as pd

revs = pd.read_csv(r"C:\Users\kisho\Downloads\Kishor Project\olist_order_reviews_dataset.csv")

# ids
revs["review_id"] = revs["review_id"].astype(str).str.strip()
revs["order_id"]  = revs["order_id"].astype(str).str.strip()

# numeric
revs["review_score"] = pd.to_numeric(revs["review_score"], errors="coerce").astype("Int64")

# comments (keep NaN as NULL in DB, just trim if present)
for c in ["review_comment_title", "review_comment_message"]:
    if c in revs.columns:
        revs[c] = revs[c].astype("string").str.strip()

# timestamps → ISO 24h
for c in ["review_creation_date", "review_answer_timestamp"]:
    if c in revs.columns:
        revs[c] = pd.to_datetime(revs[c], errors="coerce")
        revs[c] = revs[c].dt.strftime("%Y-%m-%d %H:%M:%S")

# PK hygiene + FK integrity
revs = revs.dropna(subset=["review_id","order_id"]).drop_duplicates(subset=["review_id"])

exist_orders = set(pd.read_sql("SELECT order_id FROM Orders", con=engine)["order_id"])
revs = revs[revs["order_id"].isin(exist_orders)]

# load
revs.to_sql("Reviews", con=engine, if_exists="append", index=False)
print("Reviews loaded:", len(revs))


Reviews loaded: 99173


In [29]:
%%sql sqlite:///olist_store.db
SELECT COUNT(*) AS rows FROM Reviews;
SELECT *
FROM Reviews
LIMIT 5;


review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,None,None,2018-01-18 00:00:00,2018-01-18 21:46:59
80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,None,None,2018-03-10 00:00:00,2018-03-11 03:05:13
228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,None,None,2018-02-17 00:00:00,2018-02-18 14:36:24
e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,None,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,None,Parabéns lojas lannister adorei comprar pela Internet seguro e prático Parabéns a todos feliz Páscoa,2018-03-01 00:00:00,2018-03-02 10:26:53


In [30]:
#### Created Order_Items Table

In [31]:
%%sql sqlite:///olist_store.db

DROP TABLE IF EXISTS Order_Items;

CREATE TABLE Order_Items (
    order_id TEXT,
    order_item_id INTEGER,
    product_id TEXT,
    seller_id TEXT,
    shipping_limit_date TEXT,
    price REAL,
    freight_value REAL,
    PRIMARY KEY (order_id, order_item_id),
    FOREIGN KEY (order_id) REFERENCES Orders(order_id),
    FOREIGN KEY (product_id) REFERENCES Products(product_id),
    FOREIGN KEY (seller_id) REFERENCES Sellers(seller_id)
);

++
||
++
++

In [32]:
### 6. Order Items – `olist_order_items_dataset.csv` → `Order_Items` table
"""
**Goal:** Create a line-item table that links orders to products and sellers, with pricing and shipping deadlines.

**Steps:**
- Read the CSV into pandas.
- Cleaned ID fields:
  - `order_id`, `product_id`, `seller_id` → stripped strings.
- Converted `shipping_limit_date` to datetime and then to ISO string format `YYYY-MM-DD HH:MM:SS`.
- Cast numeric fields:
  - `order_item_id`, `price`, `freight_value` → numeric (`to_numeric` with `errors='coerce'`).
- Dropped rows missing any part of the composite key or required foreign keys (`order_id`, `order_item_id`, `product_id`, `seller_id`).
- Enforced foreign key integrity:
  - Kept only rows where `order_id` exists in `Orders`.
  - Kept only rows where `product_id` exists in `Products`.
  - Kept only rows where `seller_id` exists in `Sellers`.
- Removed duplicates on the composite key (`order_id`, `order_item_id`).
- Loaded the cleaned data into the `Order_Items` table.
"""

"\n**Goal:** Create a line-item table that links orders to products and sellers, with pricing and shipping deadlines.\n\n**Steps:**\n- Read the CSV into pandas.\n- Cleaned ID fields:\n  - `order_id`, `product_id`, `seller_id` → stripped strings.\n- Converted `shipping_limit_date` to datetime and then to ISO string format `YYYY-MM-DD HH:MM:SS`.\n- Cast numeric fields:\n  - `order_item_id`, `price`, `freight_value` → numeric (`to_numeric` with `errors='coerce'`).\n- Dropped rows missing any part of the composite key or required foreign keys (`order_id`, `order_item_id`, `product_id`, `seller_id`).\n- Enforced foreign key integrity:\n  - Kept only rows where `order_id` exists in `Orders`.\n  - Kept only rows where `product_id` exists in `Products`.\n  - Kept only rows where `seller_id` exists in `Sellers`.\n- Removed duplicates on the composite key (`order_id`, `order_item_id`).\n- Loaded the cleaned data into the `Order_Items` table.\n"

In [33]:
items = pd.read_csv(r"C:\Users\kisho\Downloads\Kishor Project\olist_order_items_dataset.csv")

# Clean string columns
items["order_id"] = items["order_id"].astype(str).str.strip()
items["product_id"] = items["product_id"].astype(str).str.strip()
items["seller_id"] = items["seller_id"].astype(str).str.strip()

# Convert datetime
items["shipping_limit_date"] = pd.to_datetime(items["shipping_limit_date"], errors="coerce")
items["shipping_limit_date"] = items["shipping_limit_date"].dt.strftime("%Y-%m-%d %H:%M:%S")

# Numeric cleaning
items["order_item_id"] = pd.to_numeric(items["order_item_id"], errors="coerce")
items["price"] = pd.to_numeric(items["price"], errors="coerce")
items["freight_value"] = pd.to_numeric(items["freight_value"], errors="coerce")

# Drop invalid or duplicates
items = items.dropna(subset=["order_id", "product_id", "seller_id"])
items = items.drop_duplicates(subset=["order_id", "order_item_id"])

# FK integrity check — keep only known order IDs
existing_orders = set(pd.read_sql("SELECT order_id FROM Orders", con=engine)["order_id"])
items = items[items["order_id"].isin(existing_orders)]

# Load into SQLite
items.to_sql("Order_Items", con=engine, if_exists="append", index=False)
print("Order_Items loaded:", len(items))


Order_Items loaded: 112650


In [34]:

%%sql sqlite:///olist_store.db
SELECT COUNT(*) AS total_rows FROM Order_Items;
SELECT * FROM Order_Items LIMIT 5;


order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.9,13.29
00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.9,19.93
000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.0,17.87
00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.9,18.14


In [35]:
#### Created Sellers Table

In [36]:
%%sql sqlite:///olist_store.db

DROP TABLE IF EXISTS Sellers;

CREATE TABLE Sellers (
  seller_id TEXT PRIMARY KEY,
  seller_zip_code_prefix TEXT,   -- keep TEXT (some datasets have non-numeric codes)
  seller_city TEXT,
  seller_state CHAR(2)
);


++
||
++
++

In [37]:
### 3. Sellers – `olist_sellers_dataset.csv` → `Sellers` table
"""
**Goal:** Create a clean dimension table of sellers.
**Steps:**
- Read the CSV into pandas.
- Normalized identifiers and text fields:
  - `seller_id` → stripped string.
  - `seller_city` → title case.
  - `seller_state` → uppercase.
  - `seller_zip_code_prefix` → kept as TEXT and stripped/uppercased.
- Dropped rows with missing `seller_id` and removed duplicate seller_ids.
- Inserted the cleaned data into the `Sellers` table.
"""

'\n**Goal:** Create a clean dimension table of sellers.\n**Steps:**\n- Read the CSV into pandas.\n- Normalized identifiers and text fields:\n  - `seller_id` → stripped string.\n  - `seller_city` → title case.\n  - `seller_state` → uppercase.\n  - `seller_zip_code_prefix` → kept as TEXT and stripped/uppercased.\n- Dropped rows with missing `seller_id` and removed duplicate seller_ids.\n- Inserted the cleaned data into the `Sellers` table.\n'

In [38]:
import pandas as pd

sellers = pd.read_csv(r"C:\Users\kisho\Downloads\Kishor Project\olist_sellers_dataset.csv")

sellers["seller_id"] = sellers["seller_id"].astype(str).str.strip()
sellers["seller_city"] = sellers["seller_city"].astype(str).str.strip().str.title()
sellers["seller_state"] = sellers["seller_state"].astype(str).str.strip().str.upper()
sellers["seller_zip_code_prefix"] = sellers["seller_zip_code_prefix"].astype(str).str.strip().str.upper()

# PK hygiene
sellers = sellers.dropna(subset=["seller_id"]).drop_duplicates(subset=["seller_id"])

# load
sellers.to_sql("Sellers", con=engine, if_exists="append", index=False)
print("Sellers loaded:", len(sellers))


Sellers loaded: 3095


In [39]:
%%sql sqlite:///olist_store.db
SELECT COUNT(*) AS rows FROM Sellers;
SELECT * FROM Sellers LIMIT 5;

seller_id,seller_zip_code_prefix,seller_city,seller_state
3442f8959a84dea7ee197c632cb2df15,13023,Campinas,SP
d1b65fc7debc3361ea86b5f14c68d2e2,13844,Mogi Guacu,SP
ce3ad9de960102d0677a81f5d0bb7b2d,20031,Rio De Janeiro,RJ
c0f3eea2e14555b6faeea3dd58c1b1c3,4195,Sao Paulo,SP
51a04a8a6bdcb23deccc82b0b80742cf,12914,Braganca Paulista,SP


In [40]:
#### Created Product_Category Table

In [41]:
%%sql sqlite:///olist_store.db

DROP TABLE IF EXISTS Product_Category;

CREATE TABLE Product_Category (
  product_category_name TEXT PRIMARY KEY,
  product_category_name_english TEXT
);


++
||
++
++

In [42]:
### 4. Product Categories – `product_category_name_translation.csv` → `Product_Category` table
"""
**Goal:** Create a lookup table that translates Portuguese category names to English.

**Steps:**
- Loaded the CSV into pandas.
- Cleaned:
  - `product_category_name` → stripped and converted to lowercase (matches product file).
  - `product_category_name_english` → stripped and formatted in title case.
- Dropped rows with missing `product_category_name` and removed duplicates.
- Loaded into the `Product_Category` table, with `product_category_name` as the primary key.
"""

'\n**Goal:** Create a lookup table that translates Portuguese category names to English.\n\n**Steps:**\n- Loaded the CSV into pandas.\n- Cleaned:\n  - `product_category_name` → stripped and converted to lowercase (matches product file).\n  - `product_category_name_english` → stripped and formatted in title case.\n- Dropped rows with missing `product_category_name` and removed duplicates.\n- Loaded into the `Product_Category` table, with `product_category_name` as the primary key.\n'

In [43]:
import pandas as pd

cat = pd.read_csv(r"C:\Users\kisho\Downloads\Kishor Project\product_category_name_translation.csv")

cat["product_category_name"] = cat["product_category_name"].astype(str).str.strip().str.lower()
cat["product_category_name_english"] = cat["product_category_name_english"].astype(str).str.strip().str.title()

cat = cat.dropna(subset=["product_category_name"]).drop_duplicates(subset=["product_category_name"])

cat.to_sql("Product_Category", con=engine, if_exists="append", index=False)
print("Product_Category loaded:", len(cat))


Product_Category loaded: 71


In [44]:
%%sql sqlite:///olist_store.db
SELECT COUNT(*) AS rows FROM Product_Category;
SELECT * FROM Product_Category LIMIT 5;


product_category_name,product_category_name_english
beleza_saude,Health_Beauty
informatica_acessorios,Computers_Accessories
automotivo,Auto
cama_mesa_banho,Bed_Bath_Table
moveis_decoracao,Furniture_Decor


In [45]:
%%sql sqlite:///olist_store.db

DROP TABLE IF EXISTS Products;

CREATE TABLE Products (
  product_id TEXT PRIMARY KEY,
  product_category_name TEXT,
  product_name_length INTEGER,
  product_description_length INTEGER,
  product_photos_qty INTEGER,
  product_weight_g REAL,
  product_length_cm REAL,
  FOREIGN KEY (product_category_name) REFERENCES Product_Category(product_category_name)
);


++
||
++
++

In [46]:
### 5. Products – `olist_products_dataset.csv` → `Products` table
"""
**Goal:** Build a product dimension table with numeric attributes and a foreign key to category.

**Challenges:** In the original CSV, some column names are misspelled (e.g., `product_name_lenght`, `product_description_lenght`).
**Steps:**
- Read the CSV using `read_csv`, treating `"None"`, `"none"`, `"NULL"`, and `""` as missing values (`na_values`).
- Normalized/renamed header names to match the database schema:
  - Renamed `product_name_lenght` / `product_name_lengh` → `product_name_length`.
  - Renamed `product_description_lenght` / `product_description_lengh` → `product_description_length`.
- Cleaned string fields:
  - `product_id` → stripped string.
  - `product_category_name` → stripped, lowercase to match `Product_Category`.
- Converted numeric fields to proper numeric types:
  - `product_name_length`, `product_description_length`, `product_photos_qty` → converted to numeric and stored as nullable Int64.
  - `product_weight_g`, `product_length_cm` → converted to numeric (float).
- Dropped rows with missing `product_id` and removed duplicate products based on `product_id`.
- Enforced foreign key consistency by keeping only products whose `product_category_name` exists in `Product_Category` (or is null).
- Subset the DataFrame to the columns defined in the `Products` table and inserted into the database.
"""


'\n**Goal:** Build a product dimension table with numeric attributes and a foreign key to category.\n\n**Challenges:** In the original CSV, some column names are misspelled (e.g., `product_name_lenght`, `product_description_lenght`).\n**Steps:**\n- Read the CSV using `read_csv`, treating `"None"`, `"none"`, `"NULL"`, and `""` as missing values (`na_values`).\n- Normalized/renamed header names to match the database schema:\n  - Renamed `product_name_lenght` / `product_name_lengh` → `product_name_length`.\n  - Renamed `product_description_lenght` / `product_description_lengh` → `product_description_length`.\n- Cleaned string fields:\n  - `product_id` → stripped string.\n  - `product_category_name` → stripped, lowercase to match `Product_Category`.\n- Converted numeric fields to proper numeric types:\n  - `product_name_length`, `product_description_length`, `product_photos_qty` → converted to numeric and stored as nullable Int64.\n  - `product_weight_g`, `product_length_cm` → converted to

In [47]:
import pandas as pd

# Load with explicit NA values
prods = pd.read_csv(
    r"C:\Users\kisho\Downloads\Kishor Project\olist_products_dataset.csv",
    na_values=["None", "none", "NULL", ""],
    keep_default_na=True
)

# Rename columns (your file uses misspellings)
prods = prods.rename(columns=str.strip)
prods = prods.rename(columns={
    "product_name_lenght": "product_name_length",
    "product_name_lengh": "product_name_length",
    "product_description_lenght": "product_description_length",
    "product_description_lengh": "product_description_length"
})

prods["product_id"] = prods["product_id"].astype(str).str.strip()
prods["product_category_name"] = prods["product_category_name"].astype(str).str.strip().str.lower()

# Convert numeric columns
for c in ["product_name_length", "product_description_length", "product_photos_qty"]:
    if c in prods.columns:
        prods[c] = pd.to_numeric(prods[c], errors="coerce").astype("Int64")

for c in ["product_weight_g", "product_length_cm"]:
    if c in prods.columns:
        prods[c] = pd.to_numeric(prods[c], errors="coerce")

# Drop duplicates / invalid keys
prods = prods.dropna(subset=["product_id"]).drop_duplicates(subset=["product_id"])

# Keep only valid categories (foreign key integrity)
existing_cats = set(pd.read_sql("SELECT product_category_name FROM Product_Category", con=engine)["product_category_name"])
prods = prods[prods["product_category_name"].isna() | prods["product_category_name"].isin(existing_cats)]

# Match DB schema
cols = [
    "product_id",
    "product_category_name",
    "product_name_length",
    "product_description_length",
    "product_photos_qty",
    "product_weight_g",
    "product_length_cm"
]
prods = prods[[c for c in cols if c in prods.columns]]

# Load to DB
prods.to_sql("Products", con=engine, if_exists="append", index=False)
print("Products loaded:", len(prods))


Products loaded: 32328


In [48]:
%%sql sqlite:///olist_store.db
SELECT COUNT(*) AS rows FROM Products;
SELECT * FROM Products LIMIT 10;


product_id,product_category_name,product_name_length,product_description_length,product_photos_qty,product_weight_g,product_length_cm
1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40,287,1,225.0,16.0
3aa071139cb16b67ca9e5dea641aaa2f,artes,44,276,1,1000.0,30.0
96bd76ec8810374ed1b65e291975717f,esporte_lazer,46,250,1,154.0,18.0
cef67bcfe19066a932b7673e239eb23d,bebes,27,261,1,371.0,26.0
9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37,402,4,625.0,20.0
41d3672d4792049fa1779bb35283ed13,instrumentos_musicais,60,745,1,200.0,38.0
732bd381ad09e530fe0a5f457d81becb,cool_stuff,56,1272,4,18350.0,70.0
2548af3e6e77a690cf3eb6368e9ab61e,moveis_decoracao,56,184,2,900.0,40.0
37cc742be07708b53a98702e77a21a02,eletrodomesticos,57,163,1,400.0,27.0
8c92109888e8cdf9d66dc7e463025574,brinquedos,36,1156,1,600.0,17.0


In [49]:
#### Created Payments Table

In [50]:
%%sql sqlite:///olist_store.db

DROP TABLE IF EXISTS Payments;

CREATE TABLE Payments (
  order_id TEXT NOT NULL,
  payment_sequential INTEGER NOT NULL, 
  payment_type TEXT,     
  payment_installments INTEGER,   
  payment_value REAL,  
  PRIMARY KEY (order_id, payment_sequential),
  FOREIGN KEY (order_id) REFERENCES Orders(order_id)
);


++
||
++
++

In [51]:

### 7. Payments – `olist_order_payments_dataset.csv` → `Payments` table
"""
**Goal:** Store one or more payment records per order (including installments and amounts).

**Steps:**
- Loaded the CSV.
- Cleaned:
  - `order_id` → stripped string.
  - `payment_type` → stripped and lowercased (e.g., `credit_card`, `boleto`).
- Converted numeric fields:
  - `payment_sequential`, `payment_installments` → numeric and cast to nullable Int64.
  - `payment_value` → numeric float.
- Dropped rows with missing `order_id` or `payment_sequential`.
- Removed duplicates on the composite key (`order_id`, `payment_sequential`).
- Enforced foreign key integrity by keeping only payments for orders that exist in the `Orders` table.
- Loaded the result into the `Payments` table.
"""

'\n**Goal:** Store one or more payment records per order (including installments and amounts).\n\n**Steps:**\n- Loaded the CSV.\n- Cleaned:\n  - `order_id` → stripped string.\n  - `payment_type` → stripped and lowercased (e.g., `credit_card`, `boleto`).\n- Converted numeric fields:\n  - `payment_sequential`, `payment_installments` → numeric and cast to nullable Int64.\n  - `payment_value` → numeric float.\n- Dropped rows with missing `order_id` or `payment_sequential`.\n- Removed duplicates on the composite key (`order_id`, `payment_sequential`).\n- Enforced foreign key integrity by keeping only payments for orders that exist in the `Orders` table.\n- Loaded the result into the `Payments` table.\n'

In [52]:
import pandas as pd

pays = pd.read_csv(r"C:\Users\kisho\Downloads\Kishor Project\olist_order_payments_dataset.csv")

# normalize strings
pays["order_id"] = pays["order_id"].astype(str).str.strip()
pays["payment_type"] = pays["payment_type"].astype(str).str.strip().str.lower()

# numeric coercion
pays["payment_sequential"]   = pd.to_numeric(pays["payment_sequential"], errors="coerce")
pays["payment_installments"] = pd.to_numeric(pays["payment_installments"], errors="coerce")
pays["payment_value"]        = pd.to_numeric(pays["payment_value"], errors="coerce")

# basic hygiene: drop missing PK parts & duplicates
pays = pays.dropna(subset=["order_id","payment_sequential"])
pays["payment_sequential"] = pays["payment_sequential"].astype("Int64")
pays["payment_installments"] = pays["payment_installments"].astype("Int64")

pays = pays.drop_duplicates(subset=["order_id","payment_sequential"])

# FK integrity: keep only payments for orders we actually loaded
existing_orders = set(pd.read_sql("SELECT order_id FROM Orders", con=engine)["order_id"])
pays = pays[pays["order_id"].isin(existing_orders)]

# optional: filter out obviously bad negatives
pays = pays[(pays["payment_value"].isna()) | (pays["payment_value"] >= 0)]

# load
pays.to_sql("Payments", con=engine, if_exists="append", index=False)
print("Payments loaded:", len(pays))

Payments loaded: 103886


In [53]:
%%sql sqlite:///olist_store.db
SELECT COUNT(*) AS rows FROM Payments;

SELECT *
FROM Payments
limit 20;

order_id,payment_sequential,payment_type,payment_installments,payment_value
b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45
298fcdf1f73eb413e4d26d01b25bc1cd,1,credit_card,2,96.12
771ee386b001f06208a7419e4fc1bbd7,1,credit_card,1,81.16
3d7239c394a212faae122962df514ac7,1,credit_card,3,51.84
1f78449c87a54faf9e96e88ba1491fa9,1,credit_card,6,341.09
0573b5e23cbd798006520e1d5b4c6714,1,boleto,1,51.95


In [54]:
%%sql sqlite:///olist_store.db
SELECT COUNT(*) AS rows FROM Payments;

SELECT *
FROM Payments
limit 5;

order_id,payment_sequential,payment_type,payment_installments,payment_value
b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


In [ ]:
#### Which product categories generate the highest total sales revenue?

In [55]:
%%sql sqlite:///olist_store.db
SELECT 
    pc.product_category_name_english AS category,
    ROUND(SUM(oi.price), 2)          AS total_revenue,
    COUNT(DISTINCT o.order_id)       AS num_orders,
    COUNT(*)                         AS num_items
FROM Orders o
JOIN Order_Items oi   ON o.order_id = oi.order_id
JOIN Products p       ON oi.product_id = p.product_id
LEFT JOIN Product_Category pc 
       ON p.product_category_name = pc.product_category_name
WHERE o.order_status = 'delivered'
GROUP BY pc.product_category_name_english
ORDER BY total_revenue DESC
LIMIT 10;


category,total_revenue,num_orders,num_items
Health_Beauty,1233131.72,8647,9465
Watches_Gifts,1166176.98,5495,5859
Bed_Bath_Table,1023434.76,9272,10953
Sports_Leisure,954852.55,7530,8431
Computers_Accessories,888724.61,6530,7644
Furniture_Decor,711927.69,6307,8160
Housewares,615628.69,5743,6795
Cool_Stuff,610204.1,3559,3718
Auto,578966.65,3810,4140
Toys,471286.48,3804,4030


In [ ]:
#### On average, how many days early or late are orders delivered in each state?

In [65]:
%%sql sqlite:///olist_store.db
SELECT 
    c.customer_state,
    ROUND(AVG(ABS(
        julianday(o.order_delivered_customer_date) 
      - julianday(o.order_estimated_delivery_date)
    )), 2) AS avg_delivery_delay_days,
    COUNT(*) AS num_delivered_orders
FROM Customers c
JOIN Orders o ON c.customer_id = o.customer_id
WHERE o.order_status = 'delivered'
  AND o.order_delivered_customer_date IS NOT NULL
  AND o.order_estimated_delivery_date IS NOT NULL
GROUP BY c.customer_state
HAVING num_delivered_orders >= 50  
ORDER BY avg_delivery_delay_days;

customer_state,avg_delivery_delay_days,num_delivered_orders
SP,11.21,40494
MS,12.13,701
DF,12.28,2080
SC,12.3,3546
ES,12.39,1995
AL,12.46,397
MA,12.81,717
TO,12.88,274
GO,13.08,1957
BA,13.21,3256


In [ ]:
#### Which customer states have the highest average order value (including freight)?”

In [57]:
%%sql sqlite:///olist_store.db
SELECT 
    c.customer_state,
    ROUND(SUM(oi.price + oi.freight_value) 
          / COUNT(DISTINCT o.order_id), 2) AS avg_order_value,
    COUNT(DISTINCT o.order_id)             AS num_orders
FROM Customers c
JOIN Orders o        ON c.customer_id = o.customer_id
JOIN Order_Items oi  ON o.order_id = oi.order_id
WHERE o.order_status = 'delivered'
GROUP BY c.customer_state
HAVING num_orders >= 50             -- optional: filter tiny states
ORDER BY avg_order_value DESC;


customer_state,avg_order_value,num_orders
PB,266.61,517
AC,244.69,80
AP,240.92,67
AL,237.21,397
RO,234.43,243
PA,224.13,946
PI,220.96,476
TO,219.0,274
RN,212.48,474
SE,209.82,335


In [ ]:
#### Which payment methods are most popular, and how much total value do they represent?”

In [59]:
%%sql sqlite:///olist_store.db
SELECT 
    p.payment_type,
    COUNT(DISTINCT p.order_id) AS num_orders,
    COUNT(*)                   AS num_payments,
    ROUND(SUM(p.payment_value), 2) AS total_paid
FROM Payments p
JOIN Orders o ON p.order_id = o.order_id
WHERE o.order_status = 'delivered'
GROUP BY p.payment_type
ORDER BY total_paid DESC;


payment_type,num_orders,num_payments,total_paid
credit_card,74304,74586,12101094.88
boleto,19191,19191,2769932.58
voucher,3679,5493,343013.19
debit_card,1485,1486,208421.12


In [60]:
#### Is there a relationship between review score and delivery delay?

In [64]:
%%sql sqlite:///olist_store.db
SELECT 
    r.review_score,
    ROUND(AVG(
        ABS(julianday(o.order_delivered_customer_date) 
          - julianday(o.order_estimated_delivery_date))
    ), 2) AS avg_delay_days,
    COUNT(*) AS num_reviews
FROM Reviews r
JOIN Orders o ON r.order_id = o.order_id
WHERE o.order_status = 'delivered'
  AND o.order_delivered_customer_date IS NOT NULL
  AND o.order_estimated_delivery_date IS NOT NULL
GROUP BY r.review_score
ORDER BY r.review_score;



review_score,avg_delay_days,num_reviews
1,12.96,9635
2,12.08,2980
3,11.82,7977
4,12.31,18911
5,12.98,56739


In [ ]:
#### Which sellers generate the highest total sales revenue?

In [63]:
%%sql sqlite:///olist_store.db
SELECT 
    s.seller_id,
    s.seller_city,
    s.seller_state,
    ROUND(SUM(oi.price), 2) AS total_revenue,
    COUNT(DISTINCT oi.order_id) AS num_orders,
    COUNT(*) AS num_items
FROM Sellers s
JOIN Order_Items oi ON s.seller_id = oi.seller_id
JOIN Orders o       ON oi.order_id = o.order_id
WHERE o.order_status = 'delivered'
GROUP BY s.seller_id, s.seller_city, s.seller_state
ORDER BY total_revenue DESC
LIMIT 10;


seller_id,seller_city,seller_state,total_revenue,num_orders,num_items
4869f7a5dfa277a7dca6462dcf3b52b2,Guariba,SP,226987.93,1124,1148
53243585a1d6dc2643021fd1853d8905,Lauro De Freitas,BA,217940.44,348,400
4a3ca9315b744ce9f8e9374361493884,Ibitinga,SP,196882.12,1772,1949
fa1c13f2614d7b5c4749cbc52fecda94,Sumare,SP,190917.14,578,579
7c67e1448b00f6e969d365cea6b010ab,Itaquaquecetuba,SP,186570.05,973,1355
7e93a43ef30c4f03f38b393420bc753a,Barueri,SP,165981.49,319,322
da8622b14eb17ae2831f4ac5b9dab84a,Piracicaba,SP,159816.87,1311,1548
7a67c85e85bb2ce8582c35f2203ad736,Sao Paulo,SP,139658.69,1145,1155
1025f0e2d44d7041d6cf58b6550e0bfa,Sao Paulo,SP,138208.56,910,1420
955fee9216a65b617aa5c0531780ce60,Sao Paulo,SP,131836.71,1261,1472
